In [16]:
from tqdm import tqdm
from referit3d.in_out.scannet_scan import ScannetScan, ScannetDataset
import pandas as pd
import string
#from extract_objs_from_description import ExtractObjsFromDescription
import glob
import numpy as np
from benchmark_auto_obj_extraction_module_sr3d import read_referring_data_scv
import multiprocessing as mp
import math
import json
from extraction_utils import *


def get3d_box_from_pcs(pc):
    """
    Given point-clouds that represent object or scene return the 3D dimension of the 3D box that contains the PCs.
    """
    w = pc[:, 0].max() - pc[:, 0].min()
    l = pc[:, 1].max() - pc[:, 1].min()
    h = pc[:, 2].max() - pc[:, 2].min()
    return w, l, h


def get3d_box_center_from_pcs(pc):
    """
    Given point-clouds that represent object or scene return the 3D center of the 3D box that contains the PCs.
    """
    w, l, h = get3d_box_from_pcs(pc)
    return np.array([pc[:, 0].max() - w / 2, pc[:, 1].max() - l / 2, pc[:, 2].max() - h / 2])


def extract_target_loc_from_pred_objs_from_description(pred_objs_list, target_class):
    indices = [c for c, x in enumerate(pred_objs_list) if x == target_class]  # find indices of the target class
    if len(indices) == 1:
        return indices[0]
    else:  # multiple targets have been found.
        # TODO: Eslam: think about a way to find which one is the target.
        # print("XXX for now will return the first occurrence")
        return indices[-1]  # for now will return the first occurrence


def scannet_loader(scan_id):
    """Helper function to load the scans in memory.
    :param scan_id:
    :return: the loaded scan.
    """
    global scannet

    # print("scan_id = ", scan_id)
    scan_i = ScannetScan(scan_id=scan_id, scannet_dataset=scannet, apply_global_alignment=False, load_dense=load_dense)
    if load_dense:
        scan_i.load_point_clouds_of_all_objects_dense()
    else:
        scan_i.load_point_clouds_of_all_objects()

    return scan_i



if __name__ == '__main__':
    df = read_referring_data_scv(file_path="nr3d_cot_ref_paraphrases.csv")
    scan_ids = df.scan_id
    # Configurations:
    # ---------------
    load_dense = False
    scannet_dataset_path = "../../scannet_dataset/"
    # scannet_dataset_path = "/media/eslam/0d208863-5cdb-4a43-9794-3ca8726831b3/3D_visual_grounding/dataset/"

    # Read the scan related information
    top_scan_dir = scannet_dataset_path + "scannet/scans"
    idx_to_semantic_class_file = '../automatic_loc_module/referit3d/data/mappings/scannet_idx_to_semantic_class.json'
    instance_class_to_semantic_class_file = '../automatic_loc_module/referit3d/data/mappings/scannet_instance_class_to_semantic_class.json'
    axis_alignment_info_file = '../automatic_loc_module/referit3d/data/scannet/scans_axis_alignment_matrices.json'
    scannet = ScannetDataset(top_scan_dir,
                             idx_to_semantic_class_file,
                             instance_class_to_semantic_class_file,
                             axis_alignment_info_file)
    # Loop on the whole scenes and load them once:
    all_scenes_paths = glob.glob(top_scan_dir+"/*")
    all_scenes_paths = list(np.unique(np.array(scan_ids)))
    scenes_dict = {}
    all_scan_ids = all_scenes_paths
    n_items = len(all_scan_ids)
    n_processes = min(mp.cpu_count(), n_items)
    pool = mp.Pool(n_processes)
    chunks = int(n_items / n_processes)

    for i, data in enumerate(pool.imap(scannet_loader, all_scan_ids, chunksize=chunks)):
        scenes_dict[all_scan_ids[i]] = data
    pool.close()
    pool.join()

    
    # Create our obj retrieval module:
    # obj_extractor = ExtractObjsFromDescription("./data/scannet_instance_class_to_semantic_class.json")
    
    with open('mapped_relation.json') as f:
        mapped_relation = json.load(f)
    relation_fn_dict = {
        'near': nearest_3dobject,
        'far': farthest_3dobject,
        'right': right_3dobject,
        'left': left_3dobject,
        'on': on_3dobject,
        'down': under_3dobject,
        'front': front_3dobject,
        'back': back_3dobject
    }
    reverse_relation_fn_dict = {
        'near': nearest_3dobject,
        'far': farthest_3dobject,
        'right': left_3dobject,
        'left': right_3dobject,
        'on': under_3dobject,
        'down': on_3dobject,
        'front': back_3dobject,
        'back': front_3dobject
    }

/home/abdelrem/3d_codes/CoT3D_VG/extract_anchors/referit3d/in_out/scannet_scan.py:97: UserWarning: scene0009_00 has some issue
  warnings.warn('{} has some issue'.format(self.scan_id))


In [17]:
import re
def get_relationship_between_2_objs(target_string, RELATIONS):
        target_string = target_string.lower()
        words_pred = []
        for rel_word in RELATIONS:
            # if rel_word in sub_phrase.split(" "):
            if re.search(r'\b%s\b' % (re.escape(rel_word.lower())), target_string) is not None:
                words_pred.append(rel_word)
        if len(words_pred) == 0:
             # search the other way around:
            for rel_word in RELATIONS:
                if re.search(r'\b%s\b' % (re.escape(target_string)), rel_word.lower()) is not None:
                    words_pred.append(rel_word)
        max_str = ''
        max_len = 0
        if len(words_pred) == 0:
            # get the closest match by finding if the target is a substring of any of the relations:
            for rel_word in RELATIONS:
                if target_string in rel_word.lower() or rel_word.lower() in target_string:
                    words_pred.append(rel_word)
        for word in words_pred:
            if len(word) > len(max_str):
                 max_str = word
        return max_str

In [18]:
sentence = 'The rear, left hand side pillow on the bed that is closest to the cabinets'
sentence = 'The pillow at the back on the right, on the bed closet to the heater.'
sentence = 'The pillow that is closest to the air conditioning unit on the wall and that is behind another pillow.' # Counter is wrong in this case.

utterance:  The rear, left hand side pillow on the bed that is closest to the cabinets
Target:  pillow
path:  ["bed", "cabinet", "pillow"]
anchor_ids:  [8, 12]
paraphrases:  []
----------------
Obj  7 :  bed 7
Obj  8 :  bed 8
Obj  12 :  cabinet 12


In [18]:
df = read_referring_data_scv(file_path="nr3d_cot_ref_paraphrases.csv")
scan_ids = df.scan_id
# Manual Sampling:
# ----------------
sentence = 'there are two boxes on the floor near the copier. both are placed in front of a closed door, but the box you are looking for is closest to the door and on top of the second box.'
#sentence = 'The pillow that is closest to the air conditioning unit on the wall and that is behind another pillow.'
#i =  df.loc[df['utterance'] == sentence].index[0]
i = 2529
print("utterance: ", df['utterance'][i])
print("Target: ", df['instance_type'][i])
print("path: ", df['path'][i])
print("anchor_ids: ", df['anchor_ids'][i])
print("paraphrases: ", df['paraphrases'][i])
scan_id = scan_ids[i]
print("----------------")
for obj_count, obj in enumerate(scenes_dict[scan_id].three_d_objects):
    if obj.instance_label in df['path'][i] and (obj.instance_label != df['instance_type'][i]):
        print("Obj ", obj_count, ": ", obj.instance_label, obj.object_id)

relations_df = pd.read_csv("relations_parsed.csv")
relations_df['used_obj'] = [False] * len(relations_df)
relations_df['used_sub'] = [False] * len(relations_df)
relations = relations_df[relations_df['id'] == i].drop_duplicates(subset=['object', 'subject', 'relation']).copy()
#print("The whole relations in the sentence are: ", relations)

gt_objs_name_all_scenes = []
gt_utternaces_all_scenes = []
pred_objs_name_all_scenes = []
counter = 0
unique_anchor_counter = 0
empty_relation_counter = 0
empty_anchor_counter = 0
no_box_counter = 0
target_mismatch_target = 0
correct_counter = 0
target_counter = 0
unique_anchor_counter2 = 0
empty_relation_counter2 = 0
empty_anchor_counter2 = 0
hard_counter = 0
empty_anchor_flag = False
empty_relation_flag = False
unique_anchor_flag = False


all_scene_anchors = {}
all_scenes_refined_path = []
all_scenes_num_anchors = []
for i in tqdm(range(len(df))):
#if True:
    scan_id = scan_ids[i]
    refined_path = json.loads(df['path'][i])
    
    # Refine the detected path:
    if len(refined_path) == 0:
        refined_path.append(df['instance_type'][i])
    if df['instance_type'][i] != refined_path[-1]:
        # The target should be the last object in the path if not force it
        if df['instance_type'][i] in refined_path:
            refined_path.remove(df['instance_type'][i])
            refined_path.append(df['instance_type'][i])
        else:  # if not exist at all, add it to the end
            refined_path.append(df['instance_type'][i])
    
    all_scenes_refined_path.append(refined_path)

    empty_anchor_flag = False
    empty_relation_flag = False
    unique_anchor_flag = False
    # Run our obj retrieval module:
    pred_objs_name = extract_objs_from_description(df.utterance[i], df.instance_type[i], refined_path)
    all_scenes_num_anchors.append(len(pred_objs_name)-1)
    # if len(pred_objs_name[-1]) >0 and  df.instance_type[i] != pred_objs_name[-1]:
    #     target_mismatch_target += 1
    trgt_idx = 9999
    # possible_anchors_dict = {}  # clear the dictionary for each scene
    all_scene_anchors[i] = [-1] * (len(pred_objs_name))

    # if len(pred_objs_name) == 1 and pred_objs_name[0] == df.instance_type[i]:
    #     # all_scene_anchors[i].append(-1)
    #     no_anchor_counter_per_scene += 1

    pred_objs_name_all_scenes.append(pred_objs_name)
    # Extract easy objs; prior knowledge (the target and the unique objects):
    pred_anchor = [None] * len(pred_objs_name)
    possible_anchors_dict = {}  # clear the dictionary for each scene
    for pred_obj_idx, pred_obj_name in enumerate(pred_objs_name):  # Loop on the predicted objects in the utterance
        counter += 1
        if '*' == pred_obj_name[0]:
            no_box_counter += 1
            continue
        # Get all the possible objects that exist in the scene and match the predicted obj class from the text:
        if not (pred_obj_name in possible_anchors_dict.keys()):
            possible_anchors_dict[pred_obj_name] = []  # initialize the list once
            for obj_3d in scenes_dict[scan_id].three_d_objects:
                if (pred_obj_name == obj_3d.instance_label): # or (pred_obj_name in obj_3d.instance_label.split()) or (obj_3d.instance_label in pred_obj_name.split()):
                    possible_anchors_dict[pred_obj_name].append(obj_3d)

        # Exclude the target:
        if pred_obj_name == df.instance_type[i]: # or pred_obj_name in df.instance_type[i] or df.instance_type[i] in pred_obj_name:
            # Detect the target location from the predicted objects from the utterance:
            obj_name = df.instance_type[i]
            if pred_obj_name != df.instance_type[i]:
                obj_name = pred_obj_name
            target_idx = extract_target_loc_from_pred_objs_from_description(pred_objs_list=pred_objs_name,
                                                                            target_class=obj_name)
            if pred_obj_idx == target_idx:  # make sure it is the target not text-distractor
                for anchor_id, anchor in enumerate(possible_anchors_dict[pred_obj_name]):
                    if anchor.object_id == df.target_id[i]:
                        target_counter += 1
                        trgt_idx = target_idx
                        target_anchor = possible_anchors_dict[pred_obj_name][anchor_id]
                        pred_anchor[pred_obj_idx] = target_anchor
                        del possible_anchors_dict[pred_obj_name][anchor_id]
                        del all_scene_anchors[i][pred_obj_idx]
                        break
                continue
        # import pdb; pdb.set_trace()
        if len(possible_anchors_dict[pred_obj_name]) == 0:
            print("XXX  Error the obj not found",pred_obj_name, "!!!")
            empty_anchor_counter += 1
            empty_anchor_flag = True
            continue
        elif len(possible_anchors_dict[pred_obj_name]) == 1:  # Unique object
            pred_anchor[pred_obj_idx] = possible_anchors_dict[pred_obj_name][0]
            all_scene_anchors[i][pred_obj_idx] = possible_anchors_dict[pred_obj_name][0].object_id
            unique_anchor_flag = True
            unique_anchor_counter += 1

    # Assign the hard objs (Several objects) using the geometry info:
    remaining_indices = [c for c, x in enumerate(pred_anchor) if x is None]  # find indices of hard objs
    objs_center = [None] * len(pred_objs_name)
    # Loop on the remaining objects:
    for idx in remaining_indices:
        hard_counter += 1
        pred_obj_name = pred_objs_name[idx]
        if pred_obj_name[0] == '*':  # Skip object that don't have a bbox
            continue

        # check unique obj may be after the target removal the obj become unique.
        if len(possible_anchors_dict[pred_obj_name]) == 1:
            pred_anchor[idx] = possible_anchors_dict[pred_obj_name][0]
            all_scene_anchors[i][idx] = possible_anchors_dict[pred_obj_name][0].object_id
            unique_anchor_flag = True
            unique_anchor_counter += 1
            continue

        # 1- Get the center of each object:
        # 1.1-unassigned objs of same class center
        unassigned_anchors_center = []
        for anchor in possible_anchors_dict[pred_obj_name]:  # unassigned objs
            obj_pc = scenes_dict[scan_id].pc[anchor.points]
            unassigned_anchors_center.append(np.mean(obj_pc, axis=0))
            # w, l, h = get3d_box_from_pcs(obj_pc)
        # 1.2-assigned objs center
        known_indices = [c for c, x in enumerate(pred_anchor) if x is not None]  # find indices of hard objs
        known_centers = []
        for known_idx in known_indices:
            obj_pc = scenes_dict[scan_id].pc[pred_anchor[known_idx].points]
            known_centers.append((get3d_box_center_from_pcs(obj_pc), pred_anchor[known_idx].instance_label))

        # 2- Get the relationship between objects:
        # get relationships containing the current obj:
        relations = get_relationship_conatining_obj(i, relations_df, pred_obj_name, target=df.instance_type[i])
        #print("relations = ", relations)
        # 1st case for random:
        if len(relations) == 0:
            # all_scene_anchors[i].append(-1)
            empty_relation_counter += 1
            empty_relation_flag = True
            rand_idx = np.random.randint(0,len(possible_anchors_dict[pred_obj_name]))
            pred_anchor[idx] = possible_anchors_dict[pred_obj_name][rand_idx]
            # del unassigned_anchors_center[rand_idx]
            all_scene_anchors[i][idx] = pred_anchor[idx].object_id
        deleted_objs = 0
        found = False
        # loop over the relations and get which one of the relations has a known object in the second value of the tuple
        for relation_tuple in relations:
            if found:
                break
            relation, obj2, flag = relation_tuple
            # Get the closest relation mapping from mapped_relation using get_closest_relation_mapping
            closest_relation = get_relationship_between_2_objs(relation, mapped_relation.keys())
            obj2_centers = [c[0] for c in known_centers if c[1] == obj2]
            if closest_relation == '' or len(obj2_centers) == 0:
                continue
            
            relation_mapped = mapped_relation[closest_relation]
            for known_center in obj2_centers: # Eslam: check is
                if len(unassigned_anchors_center) == 0:
                    break
                if flag:
                    approx_anchor = relation_fn_dict[relation_mapped](known_center, unassigned_anchors_center)
                else:
                    approx_anchor = reverse_relation_fn_dict[relation_mapped](known_center, unassigned_anchors_center)
                # get the index of the chosen center with regards to unassigned_anchors_center:
                correct_counter += 1
                if approx_anchor is not None:
                    chosen_center_idx = np.argmin(np.linalg.norm(np.array(unassigned_anchors_center) - np.array(approx_anchor), axis=1))
                    del unassigned_anchors_center[chosen_center_idx]
                    chosen_center_idx = chosen_center_idx + deleted_objs
                    deleted_objs += 1
                    found = True
                    pred_anchor[idx] = possible_anchors_dict[pred_obj_name][chosen_center_idx]
                    all_scene_anchors[i][idx] = possible_anchors_dict[pred_obj_name][chosen_center_idx].object_id
                else:
                    # the relation is not valid (e.g., no object is following the relation)
                    # So get the nearest:
                    approx_anchor = relation_fn_dict['near'](known_center, unassigned_anchors_center)
                    chosen_center_idx = np.argmin(np.linalg.norm(np.array(unassigned_anchors_center) - np.array(approx_anchor), axis=1))
                    del unassigned_anchors_center[chosen_center_idx]
                    found = True
                    chosen_center_idx = chosen_center_idx + deleted_objs
                    deleted_objs += 1
                    pred_anchor[idx] = possible_anchors_dict[pred_obj_name][chosen_center_idx]
                    all_scene_anchors[i][idx] = possible_anchors_dict[pred_obj_name][chosen_center_idx].object_id
                break
    
        # 2nd case for random:
        # If u can not find object in all relations so take the nearest one (Random guess)
        if pred_anchor[idx] == None:
            empty_relation_counter += 1
            rand_idx = np.random.randint(0,len(possible_anchors_dict[pred_obj_name]))
            pred_anchor[idx] = possible_anchors_dict[pred_obj_name][rand_idx]
            all_scene_anchors[i][idx] = pred_anchor[idx].object_id
            empty_relation_flag = True
                        
print("---------------------")
print("num of all objects: ", counter)
print("num of Target objects: ", (target_counter/counter)*100)
print("num of Unique objects: ", (unique_anchor_counter/counter)*100)
print("num of Geometry objects: ", (correct_counter/counter)*100)
print("num of Random objects: ", (empty_relation_counter/counter)*100)
print("num of No box objects: ", (no_box_counter/counter)*100)
print("num of No anchors objects: ", (empty_anchor_counter/counter)*100)
print("---------------------")
print("num of all objects: ", counter)
print("num of Target objects: ", target_counter)
print("num of Unique objects: ", unique_anchor_counter)
print("num of Geometry objects: ", correct_counter)
print("num of Random objects: ", empty_relation_counter)
print("num of No box objects: ", no_box_counter)
print("num of No anchors objects: ", empty_anchor_counter)
print("---------------------")
print("Check the total number: ", (target_counter + unique_anchor_counter + correct_counter + empty_relation_counter + no_box_counter + empty_anchor_counter)==counter)
print("---------------------")
print("The Assigned anchors are: ", len(pred_anchor))

df['path'] = all_scenes_refined_path
df['num_anchors'] = all_scenes_num_anchors

for obj_count, obj in enumerate(pred_anchor):
    print("obj = ", obj)
    if obj:  # as sometimes it is none
        print("Obj ", obj_count, ": ", obj.instance_label, obj.object_id)

  0%|          | 0/41503 [00:00<?, ?it/s]

utterance:  choose the coffee table farthest from the longest table and between three seater couch and arm chair
Target:  table
path:  ["table", "couch", "armchair", "table"]
anchor_ids:  [-1, -1, 10]
paraphrases:  ["choose the coffee table farthest from the longest table and between a three-seater sofa and an armchair", "choose the coffee table farthest from the longest table and between a three-seater sofa and an arm chair", "choose the coffee table farthest from the longest table and between a three-seat couch and an armchair", "choose the coffee table farthest from the longest table and between a three-seat couch and an arm chair", "choose the coffee table farthest from the longest table and between a three-seater couch and an armchair", "choose the coffee table farthest from the longest table and between a three-seater couch and an arm chair"]
----------------
Obj  1 :  armchair 1
Obj  2 :  couch 2
Obj  3 :  couch 3
Obj  4 :  couch 4
Obj  14 :  chair 14
Obj  15 :  chair 15
Obj  16

100%|██████████| 41503/41503 [02:19<00:00, 297.72it/s]


---------------------
num of all objects:  90543
num of Target objects:  45.83788917972676
num of Unique objects:  23.97313983411197
num of Geometry objects:  15.57381575604961
num of Random objects:  4.528235203163138
num of No box objects:  10.086920026948523
num of No anchors objects:  0.0
---------------------
num of all objects:  90543
num of Target objects:  41503
num of Unique objects:  21706
num of Geometry objects:  14101
num of Random objects:  4100
num of No box objects:  9133
num of No anchors objects:  0
---------------------
Check the total number:  True
---------------------
The Assigned anchors are:  3
obj =  <referit3d.in_out.three_d_object.ThreeDObject object at 0x147d14db3f10>
Obj  0 :  wall 14
obj =  <referit3d.in_out.three_d_object.ThreeDObject object at 0x147d14db3e90>
Obj  1 :  bookshelf 13
obj =  <referit3d.in_out.three_d_object.ThreeDObject object at 0x147d14db3f90>
Obj  2 :  chair 15


In [19]:
# Saving the new file:
anchor_ids = []
for i in range(len(df)):
    anchor_ids.append(all_scene_anchors[i])

df['anchor_ids'] = anchor_ids
df.to_csv('nr3d_cot_ref_paraphrases_num_anchors.csv')

In [31]:
with open('all_scene_anchors_normal_objs.json', 'w') as fp:
    json.dump(all_scene_anchors, fp)

In [62]:
import csv
import pandas as pd
from tqdm import tqdm
import numpy as np


def read_csv_as_list_dict(csv_pth):
    rows = []
    with open(csv_pth, 'r') as csvfile:
        for row in csv.DictReader(csvfile, delimiter=','):
            rows.append(row)
    return rows


def clean_obj_path(rows):
    print("Cleaning the objects path......")
    for row in rows:
        clean_objs = []
        print(row["path"])
        if "'" in row["path"]:
            objects = row["path"].split(',')
            for obj in objects:
                clean_objs.append(obj.strip().split("'")[1])
        row["path"] = clean_objs
    print("Finish the cleaning.")
    return rows


def get_logical_pth_lang(data_dict):
    """
    Convert the string into readable list
    """
    for idx, row in enumerate(data_dict["path"]):
        clean_objs = []
        if "'" in row:
            objects = row.split(',')
            for obj in objects:
                obj = obj.strip().split("'")[1]
                if '*' in obj:
                    obj = obj[1:]
                clean_objs.append(obj)
        data_dict["path"][idx] = clean_objs

    return data_dict


def clean_paraphrased(data_dict):
    """
    Convert the string into readable list
    """
    for idx, row in enumerate(data_dict["paraphrases"]):
        clean_objs = []
        if '"' in row:
            objects = row.split(',')
            for obj in objects:
                obj = obj.strip().split('"')[1]
                clean_objs.append(obj)
        data_dict["paraphrases"][idx] = clean_objs

    return data_dict


def unique(list1):
    x = np.array(list1)
    return np.unique(x)

In [15]:
# Check percentages of number of anchors:
# ----------------------------------------
rows = pd.read_csv("nr3d_cot_ref_paraphrases_num_anchors.csv")
rows = get_logical_pth_lang(rows)
num_objs_per_sentences = []
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
count_5 = 0
for row in rows["path"]:
    num_objs_per_sentences.append(len(row))
for num in num_objs_per_sentences:
    if num ==1:
        count_1 +=1
    elif num ==2:
        count_2 +=1
    elif num ==3:
        count_3 +=1
    elif num ==4:
        count_4 +=1
    elif num >=5:
        count_5 +=1    
print(100*count_1/len(num_objs_per_sentences))    
print(100*count_2/len(num_objs_per_sentences))    
print(100*count_3/len(num_objs_per_sentences))    
print(100*count_4/len(num_objs_per_sentences))    
print(100*count_5/len(num_objs_per_sentences))    

/home/abdelrem/anaconda3/envs/artemis-m2/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


17.43247476086066
53.80815844637737
23.046526757101898
4.7851962508734305
0.9276437847866419


In [22]:
# Check percentages of number of anchors:
# ----------------------------------------
rows = pd.read_csv("nr3d_cot_ref_paraphrases_num_anchors.csv")
rows = clean_paraphrased(rows)
num_paraphrased_per_sentences = []
count_0 = 0
count_1 = 0
count_2 = 0
count_3 = 0
count_4 = 0
count_5 = 0
for row in rows["paraphrases"]:
    num_objs_per_sentences.append(len(row))
for num in num_objs_per_sentences:
    if num ==0:
        count_0 +=1
    elif num ==1:
        count_1 +=1
    elif num ==2:
        count_2 +=1
    elif num ==3:
        count_3 +=1
    elif num ==4:
        count_4 +=1
    elif num >=5:
        count_5 +=1    
print(100*count_0/len(num_objs_per_sentences))    
print(100*count_1/len(num_objs_per_sentences))    
print(100*count_2/len(num_objs_per_sentences))    
print(100*count_3/len(num_objs_per_sentences))    
print(100*count_4/len(num_objs_per_sentences))    
print(100*count_5/len(num_objs_per_sentences))    

/home/abdelrem/anaconda3/envs/artemis-m2/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


15.786810592005397
13.408669252825097
24.697009854709297
14.534692271241436
8.646764490920335
22.926053538298437


In [77]:
# Check how many samples have one relation:
relation_rows = pd.read_csv("relations_parsed.csv")
print("the original length is: ", len(relation_rows))
relation_rows = relation_rows.drop_duplicates(subset=['org_utterance'], keep=False)
print("the length of item that has one relation is: ", len(relation_rows))

# make the utterance lower:
rows = pd.read_csv("nr3d_cot_ref_paraphrases_num_anchors.csv")
"""
for idx, row in enumerate(rows["utterance"]):
    rows["utterance"][idx] = row.lower()
"""

# Check how many samples have one relation and one anchor only:
unique_rel_anchor_data = pd.DataFrame(columns = rows.columns)
unique_rel_anchor_counter = 0
objects, relations, subjects = [], [], []
for index, relation_row in tqdm(relation_rows.iterrows()):
    #print(relation_row['org_utterance'])
    #print(len(rows[rows.utterance.str.contains(relation_row['org_utterance'],case=False)]))
    index = int(index)
    mask = rows[rows.utterance.str.lower().isin([relation_row['org_utterance']])]
    if len(mask) == 1:
        if (mask.num_anchors.values[0] == 1) and (mask.anchor_ids.values[0][0] != -1):
            if (type(relation_row.object) == str) and (type(relation_row.relation) == str) and (type(relation_row.subject) == str):
                objects.append(relation_row.object)
                relations.append(relation_row.relation)
                subjects.append(relation_row.subject)
                unique_rel_anchor_data = unique_rel_anchor_data.append(mask)
                unique_rel_anchor_counter += 1
print(unique_rel_anchor_counter)
unique_rel_anchor_data['object'] = objects
unique_rel_anchor_data['relation'] = relations
unique_rel_anchor_data['subject'] = subjects


the original length is:  77140
the length of item that has one relation is:  16743


16743it [03:36, 77.36it/s]

11048


In [78]:
unique_rel_anchor_data.to_csv('nr3d_cot_unique_rel_anchor_data.csv')

In [81]:
import json


unique_relations = unique(relations)
print(len(unique_relations))

with open('mapped_relation.json') as f:
    mapped_relation = json.load(f)

found_c = 0
unique_rel_map_dict = {}
for uni_rel in unique_relations:
    found_flag = False
    # Exact matching:
    for mapped_rel in mapped_relation.keys():
        if mapped_rel == uni_rel.lower():
            found_rel = mapped_rel
            found_c += 1
            found_flag = True
            break
    if not found_flag:  # Not exact matching
        found_rel = []
        for mapped_rel in mapped_relation.keys():
            if mapped_rel in uni_rel.lower():
                found_rel.append(mapped_rel)
        if len(found_rel):
            found_rel = max(found_rel, key=len)
            found_c += 1
            found_flag = True
    
    # If still can't get it, assume near:
    if not found_flag:
        found_rel = "near"

    unique_rel_map_dict[uni_rel] = found_rel
print(found_c)
with open('unique_rel_map_dict.json', 'w') as fp:
    json.dump(unique_rel_map_dict, fp)

# Create another dict for opposite relations:
# -------------------------------------------
with open('mapped_relation_opposite.json') as f:
    opposite_dict = json.load(f)
for unique_rel, mapped_rel in unique_rel_map_dict.items():
    unique_rel_map_dict[unique_rel] = opposite_dict[mapped_rel]

with open('unique_rel_map_dict_opposite.json', 'w') as fp:
    json.dump(unique_rel_map_dict, fp)


1314
1074


In [8]:
import pandas as pd
d_set = pd.read_csv("/home/abdelrem/3d_codes/CoT3D_VG/extract_anchors/nr3d_cot_ref_paraphrases_num_anchors.csv")
print("File 1: ", len(d_set))
unique_rel_df = pd.read_csv("/home/abdelrem/3d_codes/CoT3D_VG/extract_anchors/nr3d_cot_unique_rel_anchor_data.csv")
print("File 2: ", len(unique_rel_df))
d_set = pd.merge(d_set, unique_rel_df, how='outer', on=['utterance'], suffixes=('', '_y'))
print(d_set.columns)
print("Merged File : ", len(d_set))
print(d_set.relation[0])
print(d_set.relation[50])

File 1:  41503
File 2:  11048
Index(['Unnamed: 0', 'Unnamed: 0.1', 'assignmentid', 'stimulus_id',
       'utterance', 'correct_guess', 'speaker_id', 'listener_id', 'scan_id',
       'instance_type', 'target_id', 'tokens', 'dataset',
       'mentions_target_class', 'uses_object_lang', 'uses_spatial_lang',
       'uses_color_lang', 'uses_shape_lang', 'path', 'anchor_ids',
       'paraphrases', 'num_anchors', 'Unnamed: 0_y', 'Unnamed: 0.1_y',
       'Unnamed: 0.1.1', 'assignmentid_y', 'stimulus_id_y', 'correct_guess_y',
       'speaker_id_y', 'listener_id_y', 'scan_id_y', 'instance_type_y',
       'target_id_y', 'tokens_y', 'dataset_y', 'mentions_target_class_y',
       'uses_object_lang_y', 'uses_spatial_lang_y', 'uses_color_lang_y',
       'uses_shape_lang_y', 'path_y', 'anchor_ids_y', 'paraphrases_y',
       'num_anchors_y', 'object', 'relation', 'subject'],
      dtype='object')
Merged File :  41503
nan
facing


In [5]:
print(d_set.relation[50])

'facing'

In [7]:
# Test Training Nr + Sr
import numpy as np
import six
import string
import random
import torch
import os
import re
import sys
import json
import argparse
import logging
import os.path as osp
import pathlib
import pathlib
import pandas as pd
from six.moves import cPickle
from six.moves import range


def load_scan_related_data(preprocessed_scannet_file, verbose=True, add_pad=True, add_no_obj=False):
    _, all_scans = unpickle_data(preprocessed_scannet_file)
    if verbose:
        print('Loaded in RAM {} scans'.format(len(all_scans)))

    instance_labels = set()
    for scan in all_scans:
        idx = np.array([o.object_id for o in scan.three_d_objects])
        instance_labels.update([o.instance_label for o in scan.three_d_objects])
        assert np.all(idx == np.arange(len(idx)))  # assert the list of objects-ids -is- the range(n_objects).
                                                   # because we use this ordering when we sample objects from a scan.
    all_scans = {scan.scan_id: scan for scan in all_scans}  # place scans in dictionary


    class_to_idx = {}
    i = 0
    for el in sorted(instance_labels):
        class_to_idx[el] = i
        i += 1

    if verbose:
        print('{} instance classes exist in these scans'.format(len(class_to_idx)))

    # Add the pad class needed for object classification
    if add_pad:
        class_to_idx['pad'] = len(class_to_idx)

    if add_no_obj:
        class_to_idx['no_obj'] = len(class_to_idx)

    scans_split = scannet_official_train_val()

    return all_scans, scans_split, class_to_idx


def read_lines(file_name):
    trimmed_lines = []
    with open(file_name) as fin:
        for line in fin:
            trimmed_lines.append(line.rstrip())
    return trimmed_lines


def scannet_official_train_val(valid_views=None, verbose=True):
    """
    :param valid_views: None or list like ['00', '01']
    :return:
    """
    pre_fix = "/home/abdelrem/3d_codes/CoT3D_VG/refering_codes/MVT-3DVG/referit3d"
    train_split = osp.join(pre_fix, 'data/scannet/splits/official/v2/scannetv2_train.txt')
    train_split = read_lines(train_split)
    test_split = osp.join(pre_fix, 'data/scannet/splits/official/v2/scannetv2_val.txt')
    test_split = read_lines(test_split)

    if valid_views is not None:
        train_split = [sc for sc in train_split if sc[-2:] in valid_views]
        test_split = [sc for sc in test_split if sc[-2:] in valid_views]

    if verbose:
        print('#train/test scans:', len(train_split), '/', len(test_split))

    scans_split = dict()
    scans_split['train'] = set(train_split)
    scans_split['test'] = set(test_split)
    return scans_split


def unpickle_data(file_name, python2_to_3=False):
    """
    Restore data previously saved with pickle_data().
    :param file_name: file holding the pickled data.
    :param python2_to_3: (boolean), if True, pickle happened under python2x, unpickling under python3x.
    :return: an generator over the un-pickled items.
    Note, about implementing the python2_to_3 see
        https://stackoverflow.com/questions/28218466/unpickling-a-python-2-object-with-python-3
    """
    in_file = open(file_name, 'rb')
    if python2_to_3:
        size = cPickle.load(in_file, encoding='latin1')
    else:
        size = cPickle.load(in_file)

    for _ in range(size):
        if python2_to_3:
            yield cPickle.load(in_file, encoding='latin1')
        else:
            yield cPickle.load(in_file)
    in_file.close()



def create_sr3d_classes_2_idx(json_pth):
    with open(json_pth) as json_file:
        data = json.load(json_file)

    instance_labels = set()
    instance_labels.update([k for k in data])

    class_to_idx = {}
    i = 0
    for el in sorted(instance_labels):
        class_to_idx[el] = i
        i += 1

    class_to_idx['pad'] = len(class_to_idx)
    class_to_idx['no_obj'] = len(class_to_idx)

    return class_to_idx



scannet_file = "/home/abdelrem/3d_codes/scannet_dataset/scannet/scan_4_nr3d_org/keep_all_points_00_view_with_global_scan_alignment/keep_all_points_00_view_with_global_scan_alignment.pkl"
predict_lang_anchors = True
anchors = "cot"
all_scans_in_dict, scans_split, class_to_idx = load_scan_related_data(scannet_file, add_no_obj=anchors != 'none' or predict_lang_anchors)
class_to_idx = create_sr3d_classes_2_idx(json_pth="referit3d/data/mappings/scannet_instance_class_to_semantic_class.json")

Loaded in RAM 707 scans
524 instance classes exist in these scans
#train/test scans: 1201 / 312


In [20]:
from ast import literal_eval


def get_logical_pth_lang(data_dict):
    """
    Convert the string into readable list
    """
    for idx, row in enumerate(data_dict["path"]):
        clean_objs = []
        if "'" in row:
            objects = row.split(',')
            for obj in objects:
                obj = obj.strip().split("'")[1]
                if '*' in obj:
                    obj = obj[1:]
                clean_objs.append(obj)
        data_dict["path"][idx] = clean_objs

    return data_dict


def decode_stimulus_string(s):
        """
        Split into scene_id, instance_label, # objects, target object id,
        distractors object id.

        :param s: the stimulus string
        """
        if len(s.split('-', maxsplit=4)) == 4:
            scene_id, instance_label, n_objects, target_id = \
                s.split('-', maxsplit=4)
            distractors_ids = ""
        else:
            scene_id, instance_label, n_objects, target_id, distractors_ids = \
                s.split('-', maxsplit=4)

        instance_label = instance_label.replace('_', ' ')
        n_objects = int(n_objects)
        target_id = int(target_id)
        distractors_ids = [int(i) for i in distractors_ids.split('-') if i != '']
        assert len(distractors_ids) == n_objects - 1

        return scene_id, instance_label, n_objects, target_id, distractors_ids


referit_csv = "/home/abdelrem/3d_codes/CoT3D_VG/extract_anchors/nr3d_cot_ref_paraphrases_num_anchors.csv"
textaug_paraphrase_percentage = 0
mentions_target_class_only = True
anchors = 'cot'
max_seq_len = 24
augment_with_sr3d = "/home/abdelrem/3d_codes/scannet_dataset/scannet/sr3d.csv"
referit_data = pd.read_csv(referit_csv)

is_nr = True if 'nr' in referit_csv else False
if is_nr and (anchors != 'none'):
    referit_data = get_logical_pth_lang(referit_data)

if textaug_paraphrase_percentage:
    referit_data = clean_paraphrased(referit_data)

if mentions_target_class_only:
    n_original = len(referit_data)
    referit_data = referit_data[referit_data['mentions_target_class']]
    referit_data.reset_index(drop=True, inplace=True)
    print('Dropping utterances without explicit '
            'mention to the target class {}->{}'.format(n_original, len(referit_data)))

keys = ['tokens', 'instance_type', 'scan_id', 'dataset', 'target_id', 'utterance', 'stimulus_id']
added_keys = ['path', 'anchor_ids', 'num_anchors', 'paraphrases'] if is_nr else ['anchors_types', 'anchor_ids']
keys += added_keys
referit_data = referit_data[keys]
referit_data.tokens = referit_data['tokens'].apply(literal_eval)

# Add the is_train data to the pandas data frame (needed in creating data loaders for the train and test)
is_train = referit_data.scan_id.apply(lambda x: x in scans_split['train'])
referit_data['is_train'] = is_train

# Trim data based on token length
train_token_lens = referit_data.tokens[is_train].apply(lambda x: len(x))
print('{}-th percentile of token length for remaining (training) data'
        ' is: {:.1f}'.format(95, np.percentile(train_token_lens, 95)))
n_original = len(referit_data)
referit_data = referit_data[referit_data.tokens.apply(lambda x: len(x) <= max_seq_len)]
referit_data.reset_index(drop=True, inplace=True)
print('Dropping utterances with more than {} tokens, {}->{}'.format(max_seq_len, n_original, len(referit_data)))

/home/abdelrem/anaconda3/envs/artemis/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Dropping utterances without explicit mention to the target class 41503->37842
95-th percentile of token length for remaining (training) data is: 24.0
Dropping utterances with more than 24 tokens, 37842->36201


In [21]:
print(len(referit_data))
print(referit_data.columns)

36201
Index(['tokens', 'instance_type', 'scan_id', 'dataset', 'target_id',
       'utterance', 'stimulus_id', 'path', 'anchor_ids', 'num_anchors',
       'paraphrases', 'is_train'],
      dtype='object')


In [22]:
# do this last, so that all the previous actions remain unchanged
if augment_with_sr3d is not None:
    print('Adding Sr3D as augmentation.')
    sr3d = pd.read_csv(augment_with_sr3d)
    sr3d.tokens = sr3d['tokens'].apply(literal_eval)
    is_train = sr3d.scan_id.apply(lambda x: x in scans_split['train'])
    sr3d['is_train'] = is_train
    sr3d = sr3d[is_train]
    # Eslam:
    keys = ['tokens', 'instance_type', 'scan_id', 'dataset', 'target_id', 'utterance', 'stimulus_id', 'anchors_types', 'anchor_ids', 'is_train']
    sr3d = sr3d[keys]
    # sr3d = sr3d[referit_data.columns]
    print('Dataset-size before augmentation:', len(referit_data))
    referit_data = pd.concat([referit_data, sr3d], axis=0)
    referit_data.reset_index(inplace=True, drop=True)
    print('Dataset-size after augmentation:', len(referit_data))
    print(referit_data.columns)

context_size = referit_data[~referit_data.is_train].stimulus_id.apply(lambda x: decode_stimulus_string(x)[2])
print('(mean) Random guessing among target-class test objects {:.4f}'.format( (1 / context_size).mean() ))

Adding Sr3D as augmentation.
Dataset-size before augmentation: 36201
Dataset-size after augmentation: 102047
Index(['anchor_ids', 'anchors_types', 'dataset', 'instance_type', 'is_train',
       'num_anchors', 'paraphrases', 'path', 'scan_id', 'stimulus_id',
       'target_id', 'tokens', 'utterance'],
      dtype='object')
(mean) Random guessing among target-class test objects 0.3814


/home/abdelrem/anaconda3/envs/artemis/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [16]:
referit_data[~referit_data.is_train].stimulus_id

TypeError: bad operand type for unary ~: 'float'

In [24]:
len(all_scans_in_dict)

707